## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [2]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

170500096/170498071 [==============================] - 11s 0us/step
(50000, 32, 32, 3)


In [4]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [5]:

classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32, kernel_size=(3,3), input_shape=(32,32,3), activation='relu'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(32, kernel_size=(3,3), activation='relu'))
classifier.add(BatchNormalization())

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(100, activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)















Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`



Epoch 1/100
50000/50000 [==============================] - 10s 204us/step - loss: 1.3370 - acc: 0.5471
Epoch 2/100
50000/50000 [==============================] - 4s 77us/step - loss: 0.8270 - acc: 0.7115
Epoch 3/100
50000/50000 [==============================] - 4s 77us/step - loss: 0.6074 - acc: 0.7884
Epoch 4/100
50000/50000 [==============================] - 4s 77us/step - loss: 0.4355 - acc: 0.8479
Epoch 5/100
50000/50000 [==============================] - 4s 77us/step - loss: 0.3025 - acc: 0.8966
Epoch 6/100
50000/50000 [==============================] - 4s 75us/step - loss: 0.2146 - acc: 0.9259
Epoch 7/100
50000/50000 [==============================] - 4s 78us/step - loss: 0.1542 - acc: 0.9472
Epoch 8/100
50000/50000 [==============================] - 4s 79us/step - loss: 0.1325 - acc: 0.9545
Epoch 9/100
50000/50000 [==============================] - 4s 79us/step - loss: 0.1225 - acc: 0.9572
Epoch 10/100
50000/50000 [==============================] - 4s 77us/step - loss: 0.1036 

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [6]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[2.1962428e-02, 5.0805449e-21, 7.2779395e-12, 9.7803760e-01,
        5.5999014e-09, 6.7590741e-13, 1.3328588e-09, 1.7643783e-08,
        2.0239173e-14, 1.1800173e-23]], dtype=float32)